# 2.5 自动微分

## 2.5.1 一个简单的例子
假设我们想对函数$y=2x^Tx$关于列向量$x$求导。首先我们创建变量$x$并为其分配一个初始值。

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算$y$关于$x$的梯度之前，我们需要一个地方来存储梯度。重要的是，我们不会在每次对一个参数求导时都分配新的内存。因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。注意，一个标量函数关于向量$x$的梯度是向量，并且与$x$具有相同的形状。

In [2]:
x.requires_grad_(True)   
# 等价于 
# x = torch.arange(4.0, requires_grad=True)
x.grad   # 默认值是None


In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

$x$是一个长度为$4$的向量，计算$x$和$x$的点积，得到了我们赋值给y的标量输出。接下来我们通过调用反向传播函数来自动计算$y$关于$x$每个分量的梯度，并打印这些梯度。

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函数$y=2x^Tx$关于$x$的梯度应为$4x$

In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

让我们计算$x$的另一个函数

In [7]:
# 在默认情况下，PyTorch会累积梯度，我们需要清楚之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 2.5.2 非标量变量的反向传播

当$y$不是标量时，向量y关于向量x导数的最自然解释是一个矩阵。对于高阶和高维的y和x，求导的结果可以是一个高阶张量。    
然而，虽然这些更奇特的对象确实会出现在高级机器学习中(包括深度学习中)，但当我们调用向量的反向计算时，我们通常会计算一批训练样本中每个组成部分的损失函数的导数。        
深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和。        

In [9]:
# 对非标量调用`backward`需要传入一个`gradient`参数，该参数指定微分函数于self的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于 y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 2.5.3 分离计算
有时我们希望将某些计算移动到记录的计算图之外。例如，假设$y$是作为$x$的函数计算的，而$z$是作为$y$和$x$的函数计算的。想象一下，我们想计算$z$关于$x$的梯度，但由于某种原因，我们希望将$y$视为一个常数，并且只考虑到$x$在$y$被计算后发挥的作用。      
在这里，我们可以分离$y$来返回一个新变量$u$，该变量与$y$具有相同的值，但丢弃计算图中如何计算$y$的任何信息。换句话说，梯度不会向后流经$u$到$x$。因此，下面的反向传播函数计算$z=u*x$关于$x$的偏导数，同时将$u$作为常数处理，而不是$z=x*x*x$关于$x$的偏导数。

In [10]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

由于记录了$y$的计算结果，我们可以随后在$y$上调用反向传播，得到$y=x*x$关于$x$的导数，即$2*x$

In [11]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## 2.5.4 Python控制流的梯度计算
即使构建函数的计算题需要通过Python控制流(例如，条件、循环或任意函数调用)，我们仍然可以计算得到变量的梯度

In [12]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad

tensor(102400.)